In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
%matplotlib inline
import seaborn as sns
# from functools import reduce

# set font
font_name = font_manager.FontProperties(
    fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
# matplotlib.rcParams['axes.unicode_minus'] = False
sns.set(font=font_name,
        rc={'axes.unicode_minus': False})  # to show negative sign on graphs

# set option
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
pd.set_option("display.precision", 2)

In [2]:
# sampling
df1 = pd.read_csv('data/20220303_0.csv')
df1 = df1.sample(frac=0.1, random_state=13)

# # merge
# df2 = pd.read_csv('data/20220303_1.csv')
# df3 = pd.read_csv('data/20220303_2.csv')
# dataframes = [df1, df2, df3]
# df = reduce(lambda left, right: pd.merge(left, right, on='고객번호', how='outer'), dataframes)

# concat
df2 = pd.read_csv('data/20220303_1.csv')
df3 = pd.read_csv('data/20220303_2.csv')
dataframes = [df1, df2, df3]
df = pd.concat(dataframes, ignore_index=True, axis=0)

# # check duplicates
# duplicated = df.duplicated(subset=None, keep='first')
# df[duplicated]

# drop duplicates
df = df.drop_duplicates(subset=None, keep='first', ignore_index=True)

# drop columns
drop_columns = [
    '직업코드',
    '직업대분류코드',
    '자택시도코드',
    '자택시군구코드',
    '자택주소',
    '소득금액구간',
    '보험료구간_1',
    '보험료구간_2',
    '보험료구간_3',
]
df = df.drop(drop_columns, axis=1)

In [3]:
print('df1 ->', df1.shape)
print('df2 ->', df2.shape)
print('df3 ->', df3.shape)
print('df ->', df.shape)

df1 -> (1278658, 20)
df2 -> (401373, 59)
df3 -> (159883, 59)
df -> (1839914, 54)


In [4]:
df.직업위험등급.unique()

array(['A', 'C', 'E', nan, 'B', 'D'], dtype=object)

In [ ]:
df.isnull().sum()

In [87]:
##### coffee1
import os
files = []
for file in os.listdir('data/소상공인시장진흥공단_상가(상권)정보_20211231'):
    if file.endswith('.csv'):
        file = os.path.join(
            'data/소상공인시장진흥공단_상가(상권)정보_20211231', file)
        files.append(file)

left = pd.DataFrame()
for file in files:
    right = pd.read_csv(file, low_memory=False)
    left = pd.concat([left, right], axis=0, ignore_index=True)
    
coffee1 = left.loc[left['상권업종중분류명']=='커피점/카페']

sb = coffee1.loc[coffee1['상호명'].str.contains('스타벅스')]
ed = coffee1.loc[coffee1['상호명'].str.contains('이디야')]
paik = coffee1.loc[coffee1['상호명'].str.contains('빽다방')]

sb = sb.groupby(['시도명', '시군구명', '시군구코드'])['상호명'].count().to_frame()
ed = ed.groupby(['시도명', '시군구명', '시군구코드'])['상호명'].count().to_frame()
paik = paik.groupby(['시도명', '시군구명', '시군구코드'])['상호명'].count().to_frame()

sb = sb.reset_index()
ed = ed.reset_index()
paik = paik.reset_index()

sb = sb.rename({'상호명':'스타벅스매장수'}, axis=1)
ed = ed.rename({'상호명':'이디야매장수'}, axis=1)
paik = paik.rename({'상호명':'빽다방매장수'}, axis=1)

coffee2 = pd.merge(sb, ed, on='시군구코드', how='outer', suffixes=('', '_drop'))
coffee2 = pd.merge(coffee2, paik, on='시군구코드', how='outer', suffixes=('', '_drop'))
coffee2.drop([col for col in coffee2.columns if 'drop' in col], axis=1, inplace=True)

In [88]:
coffee2

,시도명,시군구명,시군구코드,스타벅스매장수,이디야매장수,빽다방매장수
0,강원도,강릉시,42150,4.0,17.0,2.0
1,강원도,고성군,42820,1.0,NaN,NaN
2,강원도,동해시,42170,1.0,1.0,1.0
3,강원도,속초시,42210,4.0,9.0,NaN
4,강원도,원주시,42130,8.0,15.0,3.0
...,...,...,...,...,...,...
239,NaN,NaN,43760,NaN,1.0,NaN
240,NaN,NaN,43800,NaN,1.0,NaN
241,NaN,NaN,43740,NaN,1.0,1.0
242,NaN,NaN,43730,NaN,1.0,1.0


##### 시도명

In [89]:
# save pickle
coffee1.to_pickle('./data/coffee1.pkl')
# read pickle
coffee1 = pd.read_pickle('./data/coffee1.pkl')

# save pickle
coffee2.to_pickle('./data/coffee2.pkl')
# read pickle
coffee2 = pd.read_pickle('./data/coffee2.pkl')

In [90]:
# 시도명
시도명 = {
    '인천':'인천광역시',
    '충남':'충청남도',
    '경기':'경기도',
    '대전':'대전광역시',
    '경남':'경상남도',
    '서울':'서울특별시',
    '전남':'전라남도',
    '전북':'전라북도',
    '제주':'제주특별자치도',
    '부산':'부산광역시',
    '광주':'광주광역시',
    '울산':'울산광역시',
    '경북':'경상북도',
    '충북':'충청북도',
    '대구':'대구광역시',
    '강원':'강원도',
    }
for i in 시도명.keys():
    df.loc[df['시도명']==i, '시도명'] = 시도명[i]

################################################################
# df.loc[df['시도명']=='인천', '시도명'] = '인천광역시'
# df.loc[df['시도명']=='충남', '시도명'] = '충청남도'
# df.loc[df['시도명']=='경기', '시도명'] = '경기도'
# df.loc[df['시도명']=='대전', '시도명'] = '대전광역시'
# df.loc[df['시도명']=='경남', '시도명'] = '경상남도'
# df.loc[df['시도명']=='서울', '시도명'] = '서울특별시'
# df.loc[df['시도명']=='전남', '시도명'] = '전라남도'
# df.loc[df['시도명']=='전북', '시도명'] = '전라북도'
# df.loc[df['시도명']=='제주', '시도명'] = '제주특별자치도'
# df.loc[df['시도명']=='부산', '시도명'] = '부산광역시'
# df.loc[df['시도명']=='광주', '시도명'] = '광주광역시'
# df.loc[df['시도명']=='울산', '시도명'] = '울산광역시'
# df.loc[df['시도명']=='경북', '시도명'] = '경상북도'
# df.loc[df['시도명']=='충북', '시도명'] = '충청북도'
# df.loc[df['시도명']=='대구', '시도명'] = '대구광역시'
# df.loc[df['시도명']=='강원', '시도명'] = '강원도'
################################################################

# 세종특별자치시
sejong = coffee1.loc[coffee1['시도명'].str.startswith('세종'), ['행정동명', '법정동명']]
sejong_행정동명 = set(sejong['행정동명'])
sejong_법정동명 = set(sejong['법정동명'])
sejong_읍면동 = sejong_행정동명|sejong_법정동명
df.loc[(df['읍면동'].isin(sejong_행정동명)) & (df['시도명']=='충청남도'), ['시도명', '시군구명']] = '세종특별자치시'
df.loc[(df['읍면동'].isin(sejong_법정동명)) & (df['시도명']=='충청남도'), ['시도명', '시군구명']] = '세종특별자치시'
# set(df.loc[df['읍면동'].isin(sejong_법정동명), '시도명']) -> # {'강원', '경남', '경북', '부산', '충남'}

##### 시군구명

In [91]:
temp1 = df.groupby(['시군구명', '시도명']).sum().reset_index()
temp2 = temp1.groupby('시군구명').count()
temp2.loc[temp2['시도명'].values>1, '시도명'].sort_values(ascending=False)
# 시군구명
# 동구     6
# 중구     6
# 남구     5
# 서구     5
# 북구     4
# 강서구    2
# 고성군    2

# set_시군구명 = set(df['시군구명']) - set(coffee['시군구명'])
# {'연기군', '청원군', '부천시 원미구', '마산시', '당진군', '부천시 소사구', '마산시 회원구', '강남', '양주군', '진구', '부천시 오정구', '강북', '광주군', '용인시', '북제주군', '남제주군', '과천시 원미구', '여주군', '천안시', '고양시 일산구', '화성군'}

################################################################
# # list_시군구명 = []
# # for i in set_시군구명:
# #     list_시군구명.append(i.split(' ')[0])
    
# # str_시군구명 = '|'.join(list_시군구명)
# # # '연기군|청원군|부천시 원미구|마산시|당진군|부천시 소사구|마산시 회원구|강남|양주군|진구|부천시 오정구|강북|광주군|용인시|북제주군|남제주군|과천시 원미구|여주군|천안시|고양시 일산구|화성군'
# # # '연기군|청원군|부천시|마산시|당진군|부천시|마산시|강남|양주군|진구|부천시|강북|광주군|용인시|북제주군|남제주군|과천시|여주군|천안시|고양시|화성군'

# # # coffee.loc[coffee['시군구명'].str.contains(str_시군구명), '시군구명'].unique()
# # # array(['용인시 처인구', '용인시 기흥구', '용인시 수지구', '부산진구', '강남구', '광진구', '강북구',
# #     #    '전주시 덕진구', '천안시 동남구', '천안시 서북구'], dtype=object)
################################################################

# 시군구명
df.loc[(df['시도명']=='부산광역시') & (df['시군구명'].str.contains('진구')), '시군구명'] = '부산진구'
df.loc[(df['시도명']=='서울특별시') & (df['시군구명'].str.contains('진구')), '시군구명'] = '광진구'
df.loc[(df['시도명']=='전라북도') & (df['시군구명'].str.contains('진구')), '시군구명'] = '전주시 덕진구'
df.loc[df['시군구명'].str.startswith('강남'), '시군구명'] = '강남구'
df.loc[df['시군구명'].str.startswith('강북'), '시군구명'] = '강북구'
df.loc[(df['시도명']=='천안시') & (df['읍면동'].str.startswith('유량동')), '시군구명'] = '천안시 동남구'
df.loc[(df['시도명']=='천안시') & (df['읍면동'].str.startswith('성환읍')), '시군구명'] = '천안시 서북구'

# df['시군구명'] == '고양시 일산구'
일산서구 = ['일산2동', '일산3동', '주엽2동', '대화동']
일산동구 = ['백석동', '마두2동', '장항2동']
df.loc[(df['시군구명'] == '고양시 일산구') & (df['읍면동'].isin(일산서구)), '시군구명'] = '일산서구'
df.loc[(df['시군구명'] == '고양시 일산구') & (df['읍면동'].isin(일산동구)), '시군구명'] = '일산동구'

In [92]:
# save pickle
df.to_pickle('./data/df.pkl')

# read pickle
df = pd.read_pickle('./data/df.pkl')

In [93]:
df['시도명+시군구명'] = df['시도명'] + df['시군구명']
coffee2['시도명+시군구명'] = coffee2['시도명'] + coffee2['시군구명']

In [94]:
set(df['시도명+시군구명']) - set(coffee2['시도명+시군구명'])

{'강원도삼척시',
 '강원도양구군',
 '강원도양양군',
 '강원도영월군',
 '강원도정선군',
 '강원도철원군',
 '강원도태백시',
 '강원도화천군',
 '경기도과천시 원미구',
 '경기도광주군',
 '경기도부천시 소사구',
 '경기도부천시 오정구',
 '경기도부천시 원미구',
 '경기도양주군',
 '경기도여주군',
 '경기도연천군',
 '경기도용인시',
 '경기도일산동구',
 '경기도일산서구',
 '경기도화성군',
 '경상남도거창군',
 '경상남도고성군',
 '경상남도남해군',
 '경상남도마산시',
 '경상남도마산시 회원구',
 '경상남도의령군',
 '경상남도창원시',
 '경상남도함안군',
 '경상남도함양군',
 '경상남도합천군',
 '경상북도군위군',
 '경상북도봉화군',
 '경상북도상주시',
 '경상북도영주시',
 '경상북도영천시',
 '경상북도예천군',
 '경상북도울릉군',
 '경상북도울진군',
 '경상북도의성군',
 '경상북도칠곡군',
 '인천광역시남구',
 '전라남도강진군',
 '전라남도고흥군',
 '전라남도곡성군',
 '전라남도구례군',
 '전라남도담양군',
 '전라남도보성군',
 '전라남도신안군',
 '전라남도영광군',
 '전라남도영암군',
 '전라남도완도군',
 '전라남도장성군',
 '전라남도장흥군',
 '전라남도함평군',
 '전라남도해남군',
 '전라남도화순군',
 '전라북도고창군',
 '전라북도김제시',
 '전라북도남원시',
 '전라북도무주군',
 '전라북도순창군',
 '전라북도완주군',
 '전라북도장수군',
 '전라북도정읍시',
 '전라북도진안군',
 '제주특별자치도남제주군',
 '제주특별자치도북제주군',
 '충청남도금산군',
 '충청남도당진군',
 '충청남도부여군',
 '충청남도연기군',
 '충청남도천안시',
 '충청남도태안군',
 '충청남도홍성군',
 '충청북도괴산군',
 '충청북도단양군',
 '충청북도영동군',
 '충청북도옥천군',
 '충청북도진천군',
 '충청북도청원군'}

In [95]:
df.loc[df['시도명']=='충청남도', '시군구명'].unique()

array(['예산군', '당진시', '아산시', '논산시', '서천군', '서산시', '태안군', '천안시 동남구', '청양군',
       '보령시', '천안시 서북구', '부여군', '공주시', '홍성군', '금산군', '연기군', '계룡시', '당진군',
       '천안시'], dtype=object)

In [96]:
set(coffee2['시도명+시군구명']) - set(df['시도명+시군구명'])

{nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan}

In [97]:
coffee2.loc[coffee2['시도명']=='충청남도', '시군구명'].unique()

array(['계룡시', '공주시', '논산시', '당진시', '보령시', '서산시', '서천군', '아산시', '예산군',
       '천안시 동남구', '천안시 서북구', '청양군'], dtype=object)

In [98]:
coffee2.loc[coffee2['시도명']=='제주특별자치도', '시군구명'].unique()

array(['서귀포시', '제주시'], dtype=object)

In [99]:
coffee2.loc[coffee2['시도명']=='전라남도', '시군구명'].unique()

array(['광양시', '나주시', '목포시', '무안군', '순천시', '여수시', '진도군'], dtype=object)

In [100]:
coffee2.loc[coffee2['시도명']=='인천광역시', '시군구명'].unique()

array(['강화군', '계양구', '남동구', '동구', '미추홀구', '부평구', '서구', '연수구', '옹진군', '중구'],
      dtype=object)

In [101]:
coffee2.loc[coffee2['시도명']=='경상북도', '시군구명'].unique()

array(['경산시', '경주시', '고령군', '구미시', '김천시', '문경시', '성주군', '안동시', '영덕군',
       '영양군', '청도군', '청송군', '포항시 남구', '포항시 북구'], dtype=object)

In [102]:
coffee2.loc[coffee2['시도명']=='경상남도', '시군구명'].unique()

array(['거제시', '김해시', '밀양시', '사천시', '산청군', '양산시', '진주시', '창녕군',
       '창원시 마산합포구', '창원시 마산회원구', '창원시 성산구', '창원시 의창구', '창원시 진해구', '통영시',
       '하동군'], dtype=object)

In [103]:
coffee2.loc[coffee2['시도명']=='경기도', '시군구명'].unique()

array(['가평군', '고양시 덕양구', '고양시 일산동구', '고양시 일산서구', '과천시', '광명시', '광주시',
       '구리시', '군포시', '김포시', '남양주시', '동두천시', '부천시', '성남시 분당구', '성남시 수정구',
       '성남시 중원구', '수원시 권선구', '수원시 영통구', '수원시 장안구', '수원시 팔달구', '시흥시',
       '안산시 단원구', '안산시 상록구', '안성시', '안양시 동안구', '안양시 만안구', '양주시', '양평군',
       '여주시', '오산시', '용인시 기흥구', '용인시 수지구', '용인시 처인구', '의왕시', '의정부시',
       '이천시', '파주시', '평택시', '포천시', '하남시', '화성시'], dtype=object)

In [ ]:
# save pickle
df_v1.to_pickle('./data/df_v1.pkl')

# read pickle
df_v1 = pd.read_pickle('./data/df_v1.pkl')

In [104]:
df_v1 = pd.merge(df, coffee2, on='시도명+시군구명', how='outer')
df_v1

,고객번호,성별구분,나이,직업위험등급,배우자자녀코드,우편번호,소득금액,홈페이지접속수,주피보험자여부,수익자여부,계약관계자여부,시도명_x,시군구명_x,읍면동,FP몰선호상품코드,상품분류코드_1,보험료_1,가입금액_1,보험기간_1,납입할보험료_1,계약월_1,계약유지여부_1,일시납계약여부_1,계약자계약시점연령_1,모집대면비대면여부,계약전홈페이지접속횟수_1,계약성립기간_1_2번째사이,보험금수령여부_1_2번째사이,서비스제공수_1_2번째사이,상품분류코드_2,보험료_2,가입금액_2,보험기간_2,월납보험료_2,납입할보험료_2,계약월_2,계약유지여부_2,일시납계약여부_2,계약자계약시점연령_2,계약성립기간_2_3번째사이,상품분류코드_3,보험료_3,가입금액_3,보험기간_3,계약자계약시점연령_3,계약자주피보험자동일여부_3,성립계약건수,유지계약건수,유지계약합계보험료,유지계약가입합계금액,유지계약최대납입기간,유지계약최대보험기간,전자청약건수,성립계약건수_직전1년,시도명+시군구명,시도명_y,시군구명_y,시군구코드,스타벅스매장수,이디야매장수,빽다방매장수
0,1.02e+11,2.0,40.0,A,0.0,403788.0,4.85e+07,0.0,0.0,1.0,1.0,인천광역시,부평구,갈산2동,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,인천광역시부평구,인천광역시,부평구,28237.0,9.0,22.0,6.0
1,1.02e+11,1.0,40.0,A,2.0,403811.0,1.76e+07,0.0,0.0,1.0,1.0,인천광역시,부평구,부개1동,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,인천광역시부평구,인천광역시,부평구,28237.0,9.0,22.0,6.0
2,1.02e+11,1.0,38.0,B,2.0,403751.0,0.00e+00,0.0,0.0,1.0,1.0,인천광역시,부평구,갈산2동,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,인천광역시부평구,인천광역시,부평구,28237.0,9.0,22.0,6.0
3,1.00e+11,1.0,72.0,B,1.0,21398.0,3.60e+07,0.0,1.0,1.0,1.0,인천광역시,부평구,부평문화로,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,인천광역시부평구,인천광역시,부평구,28237.0,9.0,22.0,6.0
4,1.02e+11,2.0,40.0,B,3.0,403831.0,2.40e+07,0.0,0.0,1.0,1.0,인천광역시,부평구,부평2동,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,인천광역시부평구,인천광역시,부평구,28237.0,9.0,22.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1839962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43760.0,NaN,1.0,NaN
1839963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43800.0,NaN,1.0,NaN
1839964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43740.0,NaN,1.0,1.0
1839965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43730.0,NaN,1.0,1.0


In [ ]:
# # df.loc[df['자택시군구코드'].isnull()] # 301381 rows
# df = df.dropna(subset=['자택시군구코드'], how='any')

# df = df.astype({'자택시군구코드':int})

# df = pd.merge(df, starbucks, left_on='자택시군구코드', right_on='시군구코드', how='outer')
# df = pd.merge(df, ediya, left_on='자택시군구코드', right_on='시군구코드', how='outer')

# regarding nan
# df['계약관계자여부']
# df.loc[df['계약관계자여부'].isnull(), '계약관계자여부']
# df['계약관계자여부'].value_counts() # nan -> 0 ?????
# df.loc[df['자택시군구코드']=='#'] -> nan

# # label encoding
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# df['직업위험등급'] = le.fit_transform(df['직업위험등급'])
# le.classes_

# one hot encoding
# df['직업대분류코드'].nunique() # 20 -> one-hot ?????
# df['직업위험등급'].nunique() # 5 -> one-hot ?????

# df.select_dtypes(exclude='object').fillna()
# df.astype('int')

# df = df.dropna(subset=['우편번호'], how='all') # 우편번호 결측치 drop
# df = df.loc[~df['우편번호'].str.contains('-', na=False)] # 우편번호 - 포함 drop
# # df = df.loc[df['우편번호'].str.contains('-', na=False)==False] # 우편번호 - 포함 drop
# df['우편번호'] = df['우편번호'].str.strip() # 우편번호 앞 뒤 공백 제거
# df = df.loc[df['우편번호']!=''] # 우편번호 '' drop

# df = df.astype({'우편번호':'int'})

# df = df.astype({'우편번호':'object'})

# df = df.dropna(subset=['우편번호'], how='all')

# df.astype({'우편번호':'object'})
# data = df['우편번호'].unique().tolist()

# data = df['우편번호'].unique().tolist()
# err = []
# old = []
# new = []
# etc = []
# for i in data:
#     if len(i)==4:
#         err.append(i)
#     elif len(i)==6:
#         old.append(i)
#     elif len(i)==5:
#         new.append(i)
#     else:
#         etc.append(i)
# print(
#     len(err),
#     len(old),
#     len(new),
#     len(etc)
# )

# df.loc[df.duplicated(subset=['우편번호']), '우편번호'].value_counts()[:10]

# df.fillna({'우편번호':-1}, inplace=True)